# Density Profiles at Given Times

This notebook plots a number of density profiles at given times and plots an underlying signal. Both in $\rho$ and in R.

In [ ]:
%matplotlib notebook
# from __future__ import print_function
import matplotlib
# matplotlib.use("Agg")
# from IPython.core.display import display, HTML
from ipfnpytools.getsig import getsig
import ipfnpytools.aug_read as aug_read
from ipfnpytools.closest import closest
from ipfnpytools.plot import plots
from warnings import warn
import numpy as np
from ipfnpytools.save_figure import save_figure
from ipfnpytools import rps_dump
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import ipywidgets as widgets
import reflectometry as rfl

## Parameters

Change these parameters to fit your needs

In [ ]:
# Shot number
shot = 37765

# Choose edition for reflectometry (0 for most recent)
edition = 0

# Choose profile source between 'afs' and 'dump'
source = 'afs'

# Path to dump file 
path = None 

# If chosen 'dump', state the relative path
support_signal = ('FPG', 'delRoben')

# Choose edition for the supporting signal (0 for most recent)
support_signal_edition = 0

# Magnetic equillibrium
equillibrium = "EQH"

# Number of density profiles to show in the plots
number_of_profiles = 3

## Load support signal from AFS

In [ ]:
signal = getsig(shot, support_signal[0], support_signal[1], edition=support_signal_edition)
plots(signal.time, signal.data, labels=signal.name)

## Get density profile data

In [ ]:
# Fetching data ---------------------------------------------------

time, lfs_signal, hfs_signal, lfs_area, hfs_area = rfl.fetch(shot, path if source=='dump' else None, verbose=True)

In [ ]:
draw = np.linspace(0, len(time), number_of_profiles + 2, dtype=np.int)
draw = draw[1:-1]

rin = getsig(shot, 'GQH', 'Rin')
raus = getsig(shot, 'GQH', 'Raus')

fig1 = plt.figure()
plt.subplots_adjust(wspace=0.2, hspace=0.6)

# Supporting plot with colored lines
ax1 = plt.subplot(212)
stamp = []
ax1.plot(signal.time, signal.data, color='k')
for i in draw:
    l = ax1.axvline(time[i], color=next(ax1._get_lines.prop_cycler)['color'])
    stamp.append(l)


ax2 = plt.subplot(221)

ax3 = plt.subplot(222, sharey=ax2)
ax3.tick_params(labelleft=False)

hfs = []
for i in draw:
    l, = ax2.plot(hfs_area[i], hfs_signal[i]*1e-19)
    hfs.append(l)

lfs = []
for i in draw:
    l, = ax3.plot(lfs_area[i], lfs_signal[i]*1e-19)
    lfs.append(l)


#labels & text
ax2.set_ylabel(r'$\mathrm{n_{e}\,[10^{19}\,m^{-3}]}$')
ax2.set_xlabel('HFS R [m]')
ax3.set_xlabel('LFS R [m]')
ax1.set_xlabel('Time [s]')


testing_slider = widgets.FloatSlider(
        value=time[draw[0]],
        min=time[0],
        max=time[-1],
        step=time[1]-time[0],
        description='Time:',
        disabled=False,
        continuous_update=True,
        orientation='horizontal',
        readout=True,
        readout_format='.6f',
    )


def update1(**kwargs):
    for i, ts in enumerate(kwargs.values()):
        time_index = closest(time, ts)
        hfs[i].set_xdata(hfs_area[time_index])
        lfs[i].set_xdata(lfs_area[time_index])
        stamp[i].set_xdata(2*[ts])
    fig1.canvas.draw_idle()
    
    
sliders = []
for i in range(number_of_profiles):
    sliders.append(widgets.FloatSlider(
        value=time[draw[i]],
        min=time[0],
        max=time[-1],
        step=time[1]-time[0],
        description='slider %d'%i,
        disabled=False,
        continuous_update=True,
        orientation='horizontal',
        readout=True,
        readout_format='.6f',
    ))
    
kwargs = {'p{0}'.format(i):slider for i, slider in enumerate(sliders)}

widgets.interact(update1, **kwargs);
    

#Inner vessel wall
ax2.axvspan(0., 1.045, color='#D3D3D3')
#ICRH limiter
ax3.axvspan(2.22, 5., color='#D3D3D3')

#Readjust plot limits
ax2.set_xlim(1.0, 1.2) # HFS R[m]
ax3.set_xlim(2.1, 2.3) # LFS R[m]
ax2.set_ylim(0, 3) # HFS & LFS density * 1e19

# #Separatrices
# rinr = rin.data[closest(rin.time, time[0])]
# ax1[0].axvspan(rinr, 1.35, color='#FFC0CB')

# rausr = raus.data[closest(raus.time, time[0])]
# ax1[1].axvspan(1.98, rausr, color='#FFC0CB')


In [ ]:
# Profile window limits
ax2.set_xlim(1.0, 1.2) # HFS R[m]
ax3.set_xlim(2.1, 2.3) # LFS R[m]
ax2.set_ylim(0, 3) # HFS & LFS density * 1e19

# Support signal window limits
ax1.set_xlim(0, 8)  # In x direction (time)
ax1.set_ylim(-0.1, 1)  # In y direction

In [ ]:
save_figure(fig1, "timed_profiles")

## Change coordinates to $\rho$

In [ ]:
from __future__ import print_function
import dd #To read 
# import kk_abock
import numpy as np
from ipfnpytools.trz_to_rhop import trz_to_rhop

In [ ]:
# Set the antenna Z
zl = 0.14
zh = 0.07

# Walls
inner_wall = 1.045
outer_wall = 2.22

inner_wall_rho = trz_to_rhop(3, inner_wall, zh, shot, eq=equillibrium)
outer_wall_rho = trz_to_rhop(3, outer_wall, zl, shot, eq=equillibrium)
rhol = trz_to_rhop(time, lfs_area, np.ones_like(lfs_area)*zl, shot, eq=equillibrium)
rhoh = trz_to_rhop(time, hfs_area, np.ones_like(hfs_area)*zh, shot, eq=equillibrium)

In [ ]:
fig2 = plt.figure()
plt.subplots_adjust(hspace=0.6)

# Supporting plot with colored lines
ax4 = plt.subplot(212)
stamp = []
ax4.plot(signal.time, signal.data, color='k')
for i in draw:
    l = ax4.axvline(time[i], color=next(ax4._get_lines.prop_cycler)['color'])
    stamp.append(l)


ax5 = plt.subplot(211)

hfs = []
lfs = []
for i in draw:
    l, = ax5.plot(rhoh[i], hfs_signal[i]*1e-19)
    hfs.append(l)
    l, = ax5.plot(rhol[i], lfs_signal[i]*1e-19, color=l.get_color(), linestyle='--')
    lfs.append(l)
    

#labels & text
ax5.set_ylabel(r'$\mathrm{n_{e}\,[10^{19}\,m^{-3}]}$')
ax5.set_xlabel(r'$\rho$ poloidal')
ax4.set_xlabel('Time [s]')

ax5.axvspan(0.0, 1.0, color='#FFC0CB')

ax5.set_xlim(0.9, 1.2)
ax5.set_ylim(-0.1, 3)


def update2(**kwargs):
    for i, ts in enumerate(kwargs.values()):
        time_index = closest(time, ts)
        hfs[i].set_xdata(rhoh[time_index])
        lfs[i].set_xdata(rhol[time_index])
        stamp[i].set_xdata(2*[ts])
    fig2.canvas.draw_idle()
    
    
sliders2 = []
for i in range(number_of_profiles):
    sliders2.append(widgets.FloatSlider(
        value=time[draw[i]],
        min=time[0],
        max=time[-1],
        step=time[1]-time[0],
        description='slider %d'%i,
        disabled=False,
        continuous_update=True,
        orientation='horizontal',
        readout=True,
        readout_format='.6f',
    ))
    
kwargs = {'p{0}'.format(i):slider for i, slider in enumerate(sliders2)}

widgets.interact(update2, **kwargs);

**Change the window manually, or by running the cell bellow**

In [ ]:
# rho poloidal window limits
ax5.set_xlim(0.9, 1.2) # In xx direction (rho)
ax5.set_ylim(-0.1, 3)  # In yy direction (density * 1e19)

# Support signal window limits
ax4.set_xlim(0, 8)  # In x direction (time)
ax4.set_ylim(-0.1, 1)  # In y direction

In [ ]:
save_figure(fig2, "three_plots_rho")